# EDA & Hypothesis Testing

### Context

Data Used: https://www.kaggle.com/yeanzc/telco-customer-churn-ibm-dataset

The dataset represents Telco data of customers, the services opted, their tenure, monthly charges etc., and a label indicating if the customer churned post a certain tenure.

The Data also has other features describing the "Churn Label" like "Churn Value","Churn Score","CLTV" and "Churn Reason" which can be used for analysis, but not for predicting churn. 

You can take a classification approach having "Churn Label" as the dependent variable or a regression approach having "Churn Score" as the dependent variable.

I am using the dataset not just to apply a predictive model, but to try out extensive hypothesis testing techniques and predictive modelling.
This simple dataset was chosen as it has both categorical and numerical independent variables and you have categorical and numerical dependent variables to consider.

Let's start with basic visualisation and EDA to get a hold of the data and the distribution it follows before moving on to the complex techniques.

In [ ]:
#importing packages used
import pandas as pd

In [ ]:
#importing the data file as a dataframe
df=pd.read_excel("telco_customer_churn.xlsx")
pd.set_option('display.max_columns', None) #to display all the columns of the dataframe
df.head() #viewing the first 5 rows by default

In [ ]:
df.columns

In [ ]:
target_share=(df[['Churn Label']].value_counts()*100.0/len(df)).reset_index() # This indicates that the data set has imbalanced class labels
list_tgt=list(target_share.columns)
list_tgt[-1:]=['Share']
target_share.columns =list_tgt
target_share

In [ ]:
df_churned=df[df['Churn Label']=='Yes']
df_notchurned=df[df['Churn Label']=='No']

Creating a list of columns to be used as independent features. The list is referred back during EDA.

In [ ]:
#creating a list of numerical and categorical column names by filtering on dtype
df_dtypes=df.dtypes.reset_index()
df_numeric=df_dtypes[df_dtypes[0]!='object']
df_categoric=df_dtypes[df_dtypes[0]=='object']

numeric_cols=list(df_numeric['index'])
categoric_cols=list(df_categoric['index'])

#Manually fixing mix up of numerical and categorical columns and removing the columns not used currently
numeric_cols.append('Total Charges')
numeric_cols.remove('Zip Code')
numeric_cols.remove('Churn Value')
numeric_cols.remove('CLTV')
categoric_cols.append('Zip Code')
categoric_cols.remove('Total Charges')
categoric_cols.remove('Churn Reason')
categoric_cols.remove('Lat Long')

In [ ]:
df[categoric_cols].head()

In [ ]:
df[numeric_cols].head()

# Describe and Visualize Categorical variables 
### To understand their relationship with the Churn Labels

In [ ]:
#Describing the categorical columns gives details on the unique values it has
#and the frequency of the top occuring category within each of them.
df[categoric_cols].astype('object').describe()

In [ ]:
# Columns with a constant value don't contribute to predicting different churn labels
temp=df[categoric_cols].astype('object').describe().T
temp[temp['unique']==1]

In [ ]:
temp[temp['unique']!=1]

In [ ]:
# Columns with a constant value don't contribute to predicting different churn labels
#Columns with a large number (like n>100) of categorical values are hard to visualise
categories_chk=list(temp[(temp['unique']>1) & (temp['unique']<5)].index)

In [ ]:
# Describing the data of churned and unchurned customers separately 
# can help us identify the column that differentiates for churn label with ease.

# This being a preliminary check, you have to look for imbalance in categories due to natural pattern.
# To begin with our target variable is imbalanced. 73.463% of users haven't churned as opposed to the other 26.537% 

In [ ]:
df_churned_T=df_churned[categoric_cols].astype('object').describe().T
df_churned_T['share']=df_churned_T['freq']*100.0/df_churned_T['count']
df_churned_T.columns=[x+'_churned' for x in df_churned_T.columns]
df_churned_T

In [ ]:
df_notchurned_T=df_notchurned[categoric_cols].astype('object').describe().T
df_notchurned_T['share']=df_notchurned_T['freq']*100.0/df_notchurned_T['count']
df_notchurned_T

In [ ]:
pd.concat([df_notchurned_T[['top','share']], df_churned_T[['top_churned','share_churned']]], axis=1)

#### Note:
The above table can be used to find out distinguishing features that can explain the variability in predicting churn.

The "top" column represents the most frequent value in a column (refer index here) and the share out of all unchurned  instances as "share" for unchurned customers.
The "top_churned" column represents the most frequent value in a column (refer index here) and the share out of all churned instances as "share_churned" for churned customers.

There are two steps involved in identifying the significant columns/features.
1. Compare the top and top_churned columns, if they are different and if share_churned has notably different value the feature is contrasting well enough between churned and unchurned instances.
Eg: Partner, Internet Service, Online Backup, Streaming TV, Streaming Movies, Payment Method
2.  Compare the top and top_churned columns, if they are same and if share_churned has notably different value the feature is contrasting well enough between churned and unchurned instances.
Eg: Senior Citizen, Dependents, Online Security, Device Protection,  Tech Support,, Contract, Paperless Billing
3. Compare the top and top_churned columns, if they are different and if share_churned is indifferent the feature is contrasting well enough between churned and unchurned instances.
Eg: Gender, Multiple Lines
4. Compare the top and top_churned columns, if they are same and if share_churned is indifferent the feature is contrasting well enough between churned and unchurned instances.
Eg: Phone Service

Understanding a lot of variables and ones with more than 2 categorical values can be tricky using this descriptive method. Hence, let's look at how we can understand significant variables (the ones showing variability w.r.t. to target label) through visualisation.


### Let's visualize and understand the same from grouped bar graphs

Here, we are displaying grouped bar graphs to display the share of a particular categorical value in a column and how it is split within churn and not churned instances.

eg: Consider the independent variable 'Gender' having values 'Female' and 'Male'. We plot the 'Female' instances that have churned (Churn Label -> Yes) as a % of all the female instances and vice versa for not chunred instances (Churn Label -> No).

We compute the same share for all 'Male' instances and plot these side by side.

In the above descriptive statistics we saw of all the churned instances (Churn Label -> Yes), what % are 'Female','Male' etc., displaying the which value is dominant in churned users within each categorical variable.

Note that below we have a different view trying to show if a particular value within a categorical variable is skewed largely towards churn/ not.

It's important to note that the skew might have a natural imbalance due to the 73-27 split of not churned and churned instances respectively in the target label. And any share value greater or lesser than this natural threshold contributes to a skew, which gives additional information to predict the target variable with aacuracy.

This natural threshold of 73.463 for un-churned users and 26.537 for churned users are added to the graph as a horizontal line plot to visualise this better.

From the above descriptive analysis we found that Gender is an insignificant feature as it has an equal mix of both 'Male' and 'Female' within each Churn Label.

If you look at the visualisation as opposed to the inference from the above line, you can understand how the skew is represented for "Gender". This gives us a clear picture that the bar graph being approximately near the threshhold lines shows the natural skew in the Churn Label and not the variability.

Similarly, other features like  Internet Service, Tech Support, Contract, Payment Method the share of few or more values are skewed towards one of the two target labels.

Eg: Internet Service - Fiber Optic Users are skewed (bar is higher than the Churned-Ref threshold line) towards Churning (Churn Label -> Yes)

This is how you can interpret categorical variables with 2-5 values with ease using grouped bar charts. This can be used for binary or multi-classification(preferably upto 5 class labels). The limitation of the values to 5 is just for ease of interpretability from these graphs.

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

for i in categories_chk:
    if i !='Churn Label':
        temp=pd.pivot_table(df, values = 'Count', index=[i,'Churn Label'],aggfunc='count').reset_index()

        t=temp.groupby(i)['Count'].sum().reset_index()
        final=pd.merge(temp,t,on=i,suffixes=('','_all'))
        final['share']=final['Count']*100.0/final['Count_all']
        print (final)
        y_no=target_share[target_share['Churn Label']=='No']['Share']
        y_yes=target_share[target_share['Churn Label']=='Yes']['Share']
        fig = px.bar(final, x="Churn Label", y="share", color=i, barmode="group", title="Churn - "+i+" Share")

        fig.add_hline(y=float(y_no),line_color="green",annotation_text="Not Churned - Ref "+str(round(float(y_no),3)),
                      annotation_position="bottom right")
        fig.add_hline(y=float(y_yes),line_color="red",annotation_text="Churned - Ref "+str(round(float(y_yes),3)),
                      annotation_position="top right")
        fig.show()

Now that we've gone over how we can plot a bar graph and how we can interpret the same to identify important features in a small dataset. Let's understand how the importance translates while modelling another day.